# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud'
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

from scipy.signal import detrend, tukey

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [5]:
model = 'eNATL60'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig_bl'


For altimetry data

In [6]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [7]:
fresults=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [8]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)

In [9]:
print(ds[name_ssh_mod].values)

[-0.06541651  0.25607326  0.25922199 ... -0.21118741 -0.30652857
 -0.30651113]


### Spectrum

In [10]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time, ds.distance, ds[name_ssh_mod],\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)



 *** [SegmentSelection()]: Will use 23 segments with a fixed length of 100 points!
     ==> 23 selected segments out of the 240 available (requested minimum length is 70 points)

CPU times: user 4min 8s, sys: 14.9 s, total: 4min 23s
Wall time: 4min 7s


In [11]:
print(NbSeg, Nsl)

23 100


In [12]:
# Loop over the segments : plots of SSH

for k in nmp.arange(NbSeg):
    vs_s = nmp.zeros((1,Nsl))
    vs_m = nmp.zeros((1,Nsl))
    vs_s_detrend = nmp.zeros((1,Nsl))
    vs_m_detrend = nmp.zeros((1,Nsl))
    vs_s_center = nmp.zeros((1,Nsl))
    vs_m_center = nmp.zeros((1,Nsl))
    vs_s_tapper = nmp.zeros((1,Nsl))
    vs_m_tapper = nmp.zeros((1,Nsl))
    it1 = IDEDSeg[k,0]
    it2 = IDEDSeg[k,1]

    dist=ds.distance[it1:it2+1].values
    vs_s[0,:] = ds[name_ssh_sat][it1:it2+1].values
    vs_m[0,:] = ds[name_ssh_mod][it1:it2+1].values
    vs_s_detrend[0,:] = detrend(vs_s[0,:],type='linear')
    vs_m_detrend[0,:] = detrend(vs_m[0,:],type='linear')
    vs_s_center[0,:] = vs_s_detrend[0,:] - nmp.mean(vs_s_detrend[0,:])
    vs_m_center[0,:] = vs_m_detrend[0,:] - nmp.mean(vs_m_detrend[0,:])
    wdw =  tukey(Nsl,0.5)
    vs_s_tapper[0,:] = vs_s_center[0,:]*wdw
    vs_m_tapper[0,:] = vs_m_center[0,:]*wdw

    clr_sat = '#AD0000'
    clr_mod = '#008ab8'
    
    fig = plt.figure(num = 1, figsize=(15,30), facecolor='w', edgecolor='k')
    ax1 = plt.subplot(411)
    l1,=ax1.plot(dist, vs_s[0], '.-', color=clr_sat, markersize=5, 
             alpha=0.5, label=name_sat, zorder=10)
    l2,=ax1.plot(dist, vs_m[0], '.-', color=clr_mod, markersize=5, 
             alpha=0.5, label=name_mod, zorder=10)
    ax1.legend(handles = [l1,l2], bbox_to_anchor=(0.2,  0.3), ncol=1, shadow=True, fancybox=True)
    ax2 = plt.subplot(412)
    l3,=ax2.plot(dist, vs_s_detrend[0], '.-', color=clr_sat, markersize=5, 
             alpha=0.5, label=name_sat+' detrend', zorder=10)
    l4,=ax2.plot(dist, vs_m_detrend[0], '.-', color=clr_mod, markersize=5, 
             alpha=0.5, label=name_mod+' detrend', zorder=10)
    ax2.legend(handles = [l3,l4],bbox_to_anchor=(0.2,  0.7), ncol=1, shadow=True, fancybox=True)
    ax3 = plt.subplot(413)
    l5,=ax3.plot(dist, vs_s_center[0], '.-', color=clr_sat, markersize=5, 
             alpha=0.5, label=name_sat+' center', zorder=10)
    l6,=ax3.plot(dist, vs_m_center[0], '.-', color=clr_mod, markersize=5, 
             alpha=0.5, label=name_mod+' center', zorder=10)
    ax3.legend(handles = [l5,l6],bbox_to_anchor=(0.2,  0.7), ncol=1, shadow=True, fancybox=True)
    ax4 = plt.subplot(414)
    l7,=ax4.plot(dist, vs_s_tapper[0], '.-', color=clr_sat, markersize=5, 
             alpha=0.5, label=name_sat+' tapper', zorder=10)
    l8,=ax4.plot(dist, vs_m_tapper[0], '.-', color=clr_mod, markersize=5, 
             alpha=0.5, label=name_mod+' tapper', zorder=10)
    ax4.legend(handles = [l7,l8],bbox_to_anchor=(0.2, 0.9), ncol=1, shadow=True, fancybox=True)
    plt.title('Segment no '+str(k))
    plt.savefig('../plots/debug/ssh_'+name_sat+'-'+name_mod+'_segment'+str(k)+'.png')
    del fig, ax1, ax2, ax3, ax4

In [ ]:
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg, ds.distance, ds[name_ssh_mod], ds[name_ssh_sat] )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)


In [ ]:
# Blabla for the plot:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('../plots/spectrum_SSH_'+name_sat+'-'+name_mod+'.png')
